In [1]:
# Load library
import csv
import pandas as pd
import numpy as np
import time

In [ ]:
data = pd.read_csv("data/international-football-results-from-1872-to-2017.csv")
data.head()


In [8]:
import sqlite3
import pandas.io.sql as sql
con = sqlite3.connect('database.sqlite')
# table = sql.read_frame('select * from some_table', con)
table = sql.read_sql('select * from football_data', con)
table.to_csv('output_1.csv')
# table.head()
# table.to_csv('output.csv')


In [7]:

table.to_csv('output.csv')

In [125]:
# Put everything together
# 1. Read data:

# 2. Add new column named "id" and "winner"
data['id'] = data.index
data['winner'] = np.where(data.home_score > data.away_score, data.home_team, 
                                     np.where(data.home_score < data.away_score, data.away_team, 'None'))
# filter data from 2005
data_2005 = data[data['date'] > '2005-01-01'].copy()
# data_2005 = data[data['date'] > '2016-01-01'].copy()
# define my own dataframe
col_names = ['id','date', 'team_1', 'team_2', 'home_team', 'tournament', 'h_win_diff', 'h_draw', 'f_goalF_1', 
             'f_goalF_2','f_goalA_1','f_goalA_2', 'f_win_1', 'f_win_2', 'f_draw_1', 'f_draw_2','ratio_odds','avg_odds_draw','result']
my_df = pd.DataFrame(columns = col_names)

start_time = time.time()
data_len = data_2005.shape[0]
for i in range(data_len):
#     print(i)
    # Current Match
    curr_match = data_2005.iloc[i,:]
    
    # Get the head-to-head result  
    pre_df = curr_match[['id','date','tournament']].copy() #Initialize dataframe
    pre_df['team_1'] =  curr_match['home_team'] if curr_match['home_team'] < curr_match['away_team'] else curr_match['away_team'] 
    pre_df['team_2'] =  curr_match['away_team'] if curr_match['home_team'] < curr_match['away_team'] else curr_match['home_team'] 
    pre_df['home_team'] = curr_match['home_team']
    
    team_1 = pre_df['team_1']
    team_2 = pre_df['team_2']
    
    df_h2h = data[(((data.home_team == team_1) & (data.away_team == team_2)) | 
                  ((data.home_team == team_2) & (data.away_team == team_1))) & 
                  (data.date < curr_match.date)]


    count = Counter(df_h2h['winner']) #Now we get no.win and no.draw   
    
    pre_df['h_win_diff'] = count[team_1] - count[team_2]
    pre_df['h_draw'] = count['None']
    
    
    # Get the recent result of each team
    # Team 1
#     team_1 = pre_df['team_1']
    df_10  = data[((data.home_team == team_1) | (data.away_team == team_1)) & 
                 (data.date < curr_match.date)]
    
    length = df_10.shape[0]
    df_10  = df_10.iloc[length-11:length-1,:]
    
    df_10['winner'] = np.where(df_10.home_score > df_10.away_score, df_10.home_team, 
                                     np.where(df_10.home_score < df_10.away_score, df_10.away_team, 'None'))
    
    pre_df['f_goalF_1'] = df_10['home_score'].sum()
    pre_df['f_goalA_1'] = df_10['away_score'].sum()
    
    dict_result_10 = Counter(df_10['winner'])
    pre_df['f_win_1']  = dict_result_10[team_1]
    pre_df['f_draw_1'] = dict_result_10['None']
    
    # Team 2
#     team_2 = pre_df['team_2']
    df_10  = data[((data.home_team == team_2) | (data.away_team == team_2)) & 
                 (data.date < curr_match.date)]
    
    length = df_10.shape[0]
    df_10  = df_10.iloc[length-11:length-1,:]
    
    df_10['winner'] = np.where(df_10.home_score > df_10.away_score, df_10.home_team, 
                                     np.where(df_10.home_score < df_10.away_score, df_10.away_team, 'None'))
    
    pre_df['f_goalF_2'] = df_10['home_score'].sum()
    pre_df['f_goalA_2'] = df_10['away_score'].sum()
    
    dict_result_10 = Counter(df_10['winner'])
    pre_df['f_win_2']  = dict_result_10[team_2]
    pre_df['f_draw_2'] = dict_result_10['None']
    
    pre_df['result'] = np.where(curr_match.winner == team_1, 'win', 
                                     np.where(curr_match.winner == team_2, 'lose', 'draw'))
    
    
    my_df = my_df.append(pre_df)
    
end_time = time.time()
# print(time.time() - start_time)
print('Execution Time : {}'.format(end_time - start_time))
my_df.to_csv('data/data_2005.csv')

Execution Time : 1371.9029848575592


In [124]:
#  Save the file to csv
my_df.to_csv('data/data_2016.csv')

In [118]:
my_df.shape

(11958, 20)

In [120]:
data_2005.shape

(11958, 10)

In [ ]:
my_df.head()


,Unnamed: 0,id,date,team_1,team_2,home_team,tournament,h_win_diff,h_draw,f_goalF_1,f_goalF_2,f_goalA_1,f_goalA_2,f_win_1,f_win_2,f_draw_1,f_draw_2,ratio_odds,avg_odds_draw,result
0,26944,26944.0,2005-01-02,Burma,Singapore,Singapore,AFF Championship,5.0,2.0,19.0,12.0,7.0,10.0,4.0,4.0,2.0,3.0,NaN,NaN,lose
1,26945,26945.0,2005-01-03,Indonesia,Malaysia,Malaysia,AFF Championship,8.0,11.0,24.0,22.0,16.0,11.0,4.0,5.0,2.0,0.0,NaN,NaN,win
2,26946,26946.0,2005-01-08,Egypt,Uganda,Egypt,Friendly,6.0,1.0,17.0,16.0,13.0,7.0,5.0,1.0,2.0,2.0,NaN,NaN,win
3,26947,26947.0,2005-01-08,Indonesia,Singapore,Indonesia,AFF Championship,8.0,7.0,20.0,15.0,18.0,12.0,4.0,5.0,2.0,3.0,NaN,NaN,lose
4,26948,26948.0,2005-01-08,French Guyana,Jamaica,Jamaica,CFU Caribbean Cup qualification,0.0,0.0,12.0,22.0,10.0,22.0,2.0,4.0,3.0,5.0,NaN,NaN,lose


In [2]:
# Read the current data
data_2005 = pd.read_csv("data/data_2005.csv", encoding='utf-8')
data_2005.head()


,Unnamed: 0,id,date,team_1,team_2,home_team,tournament,h_win_diff,h_draw,f_goalF_1,f_goalF_2,f_goalA_1,f_goalA_2,f_win_1,f_win_2,f_draw_1,f_draw_2,ratio_odds,avg_odds_draw,result
0,26944,26944.0,2005-01-02,Burma,Singapore,Singapore,AFF Championship,5.0,2.0,19.0,12.0,7.0,10.0,4.0,4.0,2.0,3.0,NaN,NaN,lose
1,26945,26945.0,2005-01-03,Indonesia,Malaysia,Malaysia,AFF Championship,8.0,11.0,24.0,22.0,16.0,11.0,4.0,5.0,2.0,0.0,NaN,NaN,win
2,26946,26946.0,2005-01-08,Egypt,Uganda,Egypt,Friendly,6.0,1.0,17.0,16.0,13.0,7.0,5.0,1.0,2.0,2.0,NaN,NaN,win
3,26947,26947.0,2005-01-08,Indonesia,Singapore,Indonesia,AFF Championship,8.0,7.0,20.0,15.0,18.0,12.0,4.0,5.0,2.0,3.0,NaN,NaN,lose
4,26948,26948.0,2005-01-08,French Guyana,Jamaica,Jamaica,CFU Caribbean Cup qualification,0.0,0.0,12.0,22.0,10.0,22.0,2.0,4.0,3.0,5.0,NaN,NaN,lose


In [6]:
# Read the bet-odd data
data_odd= pd.read_csv("data/closing_odds.csv")
data_odd.head()

,match_id,league,match_date,home_team,home_score,away_team,away_score,avg_odds_home_win,avg_odds_draw,avg_odds_away_win,max_odds_home_win,max_odds_draw,max_odds_away_win,top_bookie_home_win,top_bookie_draw,top_bookie_away_win,n_odds_home_win,n_odds_draw,n_odds_away_win
0,170088,England: Premier League,2005-01-01,Liverpool,0,Chelsea,1,2.9944,3.1944,2.2256,3.20,3.25,2.29,Paddy Power,Sportingbet,Expekt,9,9,9
1,170089,England: Premier League,2005-01-01,Fulham,3,Crystal Palace,1,1.9456,3.2333,3.6722,2.04,3.30,4.15,Pinnacle Sports,bet-at-home,Expekt,9,9,9
2,170090,England: Premier League,2005-01-01,Aston Villa,1,Blackburn,0,1.8522,3.2611,4.0144,2.00,3.40,4.50,Pinnacle Sports,Paddy Power,Sportingbet,9,9,9
3,170091,England: Premier League,2005-01-01,Bolton,1,West Brom,1,1.6122,3.4133,5.4722,1.67,3.57,6.27,Coral,Pinnacle Sports,Pinnacle Sports,9,9,9
4,170092,England: Premier League,2005-01-01,Charlton,1,Arsenal,3,5.9878,3.4778,1.5567,7.00,3.60,1.62,Expekt,Paddy Power,bet365,9,9,9


In [27]:
# from iso3166 import countries
length = data_odd.shape[0]
# count = 0

# Initialze my_df
my_df = data_odd.iloc[1:3,:]
my_df = my_df[0:0]

for i in range(length):
    curr_match = data_odd.iloc[i,:]
    if curr_match['home_team'] in countries:
#         count = count + 1
        print(curr_match['home_team'])
        my_df = my_df.append(curr_match)
# print("Count: "+str(count))

Monaco
Monaco
Monaco
Latvia
Japan
Uzbekistan
Bahrain
Greece
Morocco
Spain
Hong Kong
Bulgaria
Algeria
Azerbaijan
Georgia
Slovakia
United Arab Emirates
Slovenia
Israel
Egypt
Malta
South Africa
Poland
Ireland
Germany
France
Italy
Monaco
Monaco
Monaco
Monaco
Kuwait
Saudi Arabia
Armenia
Bulgaria
Estonia
Liechtenstein
Croatia
Poland
Georgia
Israel
Romania
Turkey
Denmark
Belgium
Morocco
France
Italy
Australia
Slovenia
Portugal
Spain
Chile
Costa Rica
Guatemala
Mexico
Ireland
Japan
Andorra
Estonia
Croatia
Latvia
Ukraine
Georgia
Kuwait
Slovakia
Poland
Slovenia
Austria
Greece
Netherlands
San Marino
Switzerland
Hungary
Israel
Italy
USA
Panama
Monaco
Estonia
Monaco
Mexico
Monaco
Ecuador
Monaco
Monaco
Romania
Norway
Japan
Monaco
USA
Poland
France
Colombia
Finland
Uzbekistan
Bahrain
Saudi Arabia
Armenia
Azerbaijan
Bulgaria
Estonia
Sweden
Faroe Islands
Ukraine
Belarus
Turkey
Iceland
Albania
Ireland
Netherlands
Norway
San Marino
Portugal
Spain
Colombia
Ecuador
USA
Chile
Guatemala
Zimbabwe
Rwanda
Angola

Paraguay
Monaco
Bahrain
Chile
Hungary
Georgia
Luxembourg
Belarus
Latvia
Malta
San Marino
Serbia
Croatia
Netherlands
Slovakia
Sweden
Italy
Iceland
Portugal
Ghana
Cyprus
Ecuador
Peru
USA
Australia
Saudi Arabia
Switzerland
Austria
Kazakhstan
Andorra
Finland
Slovakia
Cyprus
Lithuania
Norway
Bulgaria
Turkey
Denmark
Slovenia
Iceland
Albania
Ukraine
France
Portugal
Spain
South Africa
Montenegro
Malta
Germany
Brazil
Canada
Colombia
Honduras
Peru
Monaco
Monaco
Bangladesh
Thailand
Oman
Lebanon
Palestine
Yemen
Cambodia
Slovakia
Hungary
Armenia
Belarus
Faroe Islands
Azerbaijan
Iceland
Cyprus
Liechtenstein
Romania
Denmark
Croatia
Greece
Poland
Slovenia
Belgium
Ireland
Italy
Uzbekistan
Uruguay
Argentina
Switzerland
Ecuador
Peru
Madagascar
Colombia
Mexico
Honduras
Croatia
Qatar
Kazakhstan
Azerbaijan
Ukraine
Georgia
Malta
Turkey
Denmark
Liechtenstein
Luxembourg
San Marino
Ireland
Netherlands
Sweden
Albania
Belgium
Germany
France
Sierra Leone
Japan
Estonia
United Arab Emirates
Poland
Finland
Austria
Sw

Japan
Kuwait
Hong Kong
China
Malaysia
Thailand
Honduras
Denmark
Kuwait
Egypt
Costa Rica
Belize
Monaco
San Marino
Honduras
USA
Guatemala
Nicaragua
Honduras
Singapore
Thailand
Indonesia
Yemen
Kuwait
Lebanon
United Arab Emirates
Bahrain
Panama
Mexico
Nicaragua
Costa Rica
Monaco
Honduras
Uzbekistan
Honduras
Costa Rica
Japan
Bahrain
Lithuania
Cyprus
Slovakia
Cyprus
Ukraine
Brazil
Ecuador
Cyprus
Serbia
Japan
Uzbekistan
Malta
San Marino
Ireland
Latvia
Mozambique
Libya
Angola
Iceland
Algeria
Cyprus
Estonia
Israel
Kuwait
Serbia
Egypt
Lithuania
Burkina Faso
Cameroon
South Africa
Turkey
Romania
Morocco
Greece
Austria
Germany
Switzerland
Tunisia
Belgium
Nigeria
France
Portugal
Spain
USA
Costa Rica
Peru
Colombia
Monaco
San Marino
Monaco
San Marino
Australia
Mexico
Monaco
San Marino
Qatar
Namibia
Uganda
Iceland
Saudi Arabia
Bahrain
Kuwait
Angola
Japan
Uzbekistan
Kenya
Rwanda
Armenia
Luxembourg
Togo
Sudan
Burkina Faso
Cyprus
Israel
Romania
Germany
Malta
Albania
Belgium
Ireland
Lithuania
Montenegro
Ne

Paraguay
Panama
Puerto Rico
Puerto Rico
Germany
Puerto Rico
San Marino
Israel
Netherlands
Algeria
Armenia
Kazakhstan
Andorra
Faroe Islands
Montenegro
Latvia
Romania
Sweden
Lithuania
Luxembourg
Estonia
Slovakia
Belgium
Greece
Liechtenstein
San Marino
Slovenia
France
Iceland
Portugal
China
Switzerland
Qatar
Kuwait
Uganda
Botswana
Gambia
South Africa
Sudan
Tunisia
Panama
Japan
Thailand
Poland
Canada
Morocco
Ethiopia
Mozambique
Swaziland
Zambia
Benin
Nigeria
Liberia
Egypt
Guatemala
Mexico
Jamaica
Georgia
Belarus
Bulgaria
Malta
Sweden
Turkey
Albania
Denmark
Austria
Croatia
Hungary
Netherlands
Norway
Serbia
Germany
Ireland
Switzerland
Italy
Japan
China
Estonia
Ukraine
United Arab Emirates
Poland
Qatar
Argentina
Canada
Jamaica
Mexico
Panama
India
Puerto Rico
Morocco
San Marino
Puerto Rico
Puerto Rico
Iraq
Monaco
Puerto Rico
San Marino
India
Armenia
Kazakhstan
Andorra
Georgia
Hungary
Cyprus
Luxembourg
Albania
Austria
Montenegro
Serbia
Germany
Greece
Ireland
Slovenia
Portugal
Spain
Puerto Rico


United Arab Emirates
Grenada
Saudi Arabia
Argentina
Monaco
Costa Rica
Ecuador
Brazil
Mexico
San Marino
Cuba
Mexico
Brazil
Uruguay
Mexico
Argentina
Uruguay
Argentina
San Marino
Kuwait
Qatar
Monaco
Gabon
Turkey
Estonia
Tajikistan
Comoros
Djibouti
Eritrea
United Arab Emirates
Seychelles
Uzbekistan
Oman
Swaziland
Iraq
Kuwait
Chad
Equatorial Guinea
Bahrain
Jordan
Qatar
Saudi Arabia
Lesotho
Bermuda
Dominican Republic
Argentina
Belize
Uruguay
Georgia
Guinea
Burkina Faso
Albania
Hungary
Cameroon
Cyprus
Denmark
Greece
Netherlands
Belgium
Poland
Ukraine
France
Cameroon
Morocco
Antigua and Barbuda
Suriname
Colombia
Guyana
Paraguay
Saint Lucia
Nicaragua
Guatemala
Somalia
Panama
Mexico
South Africa
Nigeria
Algeria
San Marino
Sudan
Morocco
Turkey
Brazil
Croatia
Montenegro
Ireland
Portugal
Barbados
Puerto Rico
Cayman Islands
Thailand
Singapore
Indonesia
Burundi
Lebanon
Madagascar
Kenya
Uzbekistan
Rwanda
Kuwait
Togo
Jordan
Qatar
Saudi Arabia
Mozambique
Namibia
Grenada
Haiti
Colombia
Ecuador
Honduras
N

Tunisia
Oman
Bahrain
United Arab Emirates
Zambia
South Africa
Iraq
Yemen
Gabon
Niger
Ghana
Bahrain
United Arab Emirates
Panama
Iraq
Kuwait
Zambia
South Africa
Namibia
Panama
Ghana
Monaco
United Arab Emirates
Iraq
Lebanon
Chile
Burkina Faso
Bahrain
Iraq
Guatemala
South Africa
Angola
Honduras
Costa Rica
Ghana
Mali
San Marino
Belize
Chile
Zambia
Nigeria
Costa Rica
Tunisia
Nicaragua
South Africa
Morocco
Panama
Costa Rica
Sweden
Thailand
Ghana
Niger
China
Zambia
Burkina Faso
Guatemala
Algeria
Monaco
San Marino
Honduras
Costa Rica
Canada
Sweden
Morocco
Belize
Niger
Honduras
Burkina Faso
Ethiopia
Algeria
Togo
USA
Oman
Mexico
Jordan
Qatar
Bahrain
Azerbaijan
Iraq
Ghana
South Africa
Poland
Burkina Faso
Luxembourg
Senegal
Mali
Burkina Faso
Uzbekistan
Thailand
Oman
Iraq
Jordan
Yemen
Qatar
Saudi Arabia
Honduras
Nepal
Japan
India
Azerbaijan
Croatia
Kenya
Belarus
Cyprus
Chile
Norway
Slovenia
Albania
Israel
Spain
Turkey
Malta
Greece
Iceland
Netherlands
Sweden
Belgium
Ireland
Romania
France
Portugal
Pa

France
Honduras
Costa Rica
United Arab Emirates
Switzerland
Brazil
Greece
Mexico
Gibraltar
Algeria
Hungary
Netherlands
Italy
Ecuador
Egypt
Iceland
Argentina
Uruguay
Chile
United Arab Emirates
Poland
Germany
Brazil
Colombia
Croatia
Japan
Costa Rica
Greece
Mexico
Cameroon
Estonia
Hungary
Argentina
Belgium
Spain
USA
United Arab Emirates
San Marino
France
Ghana
Portugal
Brazil
Mexico
Spain
Chile
Colombia
Uruguay
Switzerland
France
Argentina
Germany
Ghana
Belgium
Brazil
Australia
Spain
China
Cameroon
Colombia
Uruguay
Japan
Italy
Switzerland
Honduras
Argentina
Germany
Indonesia
Nigeria
Belgium
China
USA
Australia
Netherlands
Cameroon
Croatia
Costa Rica
Italy
Greece
Japan
Nigeria
Ecuador
Honduras
Indonesia
Portugal
USA
Algeria
Brazil
Colombia
Netherlands
Costa Rica
China
France
Germany
Argentina
Belgium
France
Brazil
Argentina
Netherlands
Brazil
Netherlands
Brazil
Germany
Qatar
Botswana
Uganda
Sierra Leone
Lesotho
Congo
Benin
Malawi
Rwanda
Kenya
Mozambique
Peru
Monaco
Kazakhstan
Romania
Azerb

In [28]:
my_df.to_csv('data_odd_match.csv')

In [29]:
my_df.shape

(6747, 19)

In [26]:
my_df = data_odd.iloc[1:3,:]
my_df = my_df[0:0]
my_df

,match_id,league,match_date,home_team,home_score,away_team,away_score,avg_odds_home_win,avg_odds_draw,avg_odds_away_win,max_odds_home_win,max_odds_draw,max_odds_away_win,top_bookie_home_win,top_bookie_draw,top_bookie_away_win,n_odds_home_win,n_odds_draw,n_odds_away_win


In [24]:
length

479440

In [15]:
#  Migrate data
curr_match = data_2005.iloc[30,:]
curr_match

Unnamed: 0                     26974
id                             26974
date                      2005-02-01
team_1                         Haiti
team_2           Trinidad and Tobago
home_team        Trinidad and Tobago
tournament                  Friendly
h_win_diff                        -2
h_draw                             5
f_goalF_1                         22
f_goalF_2                         21
f_goalA_1                         16
f_goalA_2                         10
f_win_1                            4
f_win_2                            7
f_draw_1                           2
f_draw_2                           0
ratio_odds                       NaN
avg_odds_draw                    NaN
result                          lose
Name: 30, dtype: object

In [16]:
data_odd[(data_odd['match_date'] == curr_match.date) & (data_odd['home_team'] == curr_match.home_team)]

,match_id,league,match_date,home_team,home_score,away_team,away_score,avg_odds_home_win,avg_odds_draw,avg_odds_away_win,max_odds_home_win,max_odds_draw,max_odds_away_win,top_bookie_home_win,top_bookie_draw,top_bookie_away_win,n_odds_home_win,n_odds_draw,n_odds_away_win


In [ ]:
import requests

req = requests.get('https://raw.githubusercontent.com/Miguel-Frazao/world-data/master/countries_data.json').json()
countries = (i['name'] for i in req)
print(list(countries))

In [18]:
from iso3166 import countries

In [20]:
'Germany U23' in countries

False

In [ ]:
data_2005['tournament'].unique()

array(['AFF Championship', 'Friendly', 'CFU Caribbean Cup qualification',
       'Cyprus International Tournament', 'FIFA World Cup qualification',
       'Lunar New Year Cup', 'UNCAF Cup', 'CFU Caribbean Cup',
       'COSAFA Cup', 'EAFF Championship', 'Baltic Cup', 'Kirin Cup',
       'Confederations Cup', 'Gold Cup', 'Amílcar Cabral Cup',
       'CECAFA Cup', 'SAFF Cup', 'AFC Asian Cup qualification',
       "King's Cup", 'African Cup of Nations',
       'Malta International Tournament', 'AFC Challenge Cup',
       'FIFA World Cup', 'UEFA Euro qualification', 'Merdeka Tournament',
       'African Cup of Nations qualification', 'Copa del Pacífico',
       'UAFA Cup qualification', 'Gulf Cup', 'WAFF Championship',
       'Copa América', 'AFC Asian Cup', 'Nehru Cup', 'South Pacific Games',
       'AFC Challenge Cup qualification', 'UEFA Euro',
       'African Nations Championship', 'VFF Cup', 'Dragon Cup',
       'ABCS Tournament', 'Nile Basin Tnmt.', 'Nations Cup',
       'Copa Paz del

In [115]:
curr_match

date          2018-03-23
home_team        Germany
away_team          Spain
home_score             1
away_score             1
tournament      Friendly
city          Düsseldorf
country          Germany
id                 38792
winner              None
Name: 38792, dtype: object

In [116]:
team_1 = 'Germany'
team_2 = 'Spain'
df_h2h = data[(((data.home_team == team_1) & (data.away_team == team_2)) | 
            ((data.home_team == team_2) & (data.away_team == team_1))) & 
            (data.date < curr_match.date)]
df_h2h

,date,home_team,away_team,home_score,away_score,tournament,city,country,id,winner
1741,1935-05-12,Germany,Spain,1,2,Friendly,Cologne,Germany,1741,Spain
1822,1936-02-23,Spain,Germany,1,2,Friendly,Barcelona,Spain,1822,Germany
2356,1942-04-12,Germany,Spain,1,1,Friendly,Berlin,Germany,2356,None
3355,1952-12-28,Spain,Germany,2,2,Friendly,Madrid,Spain,3355,None
4248,1958-03-19,Germany,Spain,2,0,Friendly,Frankfurt am Main,Germany,4248,Germany
6163,1966-07-20,Germany,Spain,2,1,FIFA World Cup,Birmingham,England,6163,Germany
7298,1970-02-11,Spain,Germany,2,0,Friendly,Seville,Spain,7298,Spain
8755,1973-11-24,Germany,Spain,2,1,Friendly,Stuttgart,Germany,8755,Germany
8820,1974-02-23,Spain,Germany,1,0,Friendly,Barcelona,Spain,8820,Spain
9607,1976-04-24,Spain,Germany,1,1,UEFA Euro qualification,Madrid,Spain,9607,None


In [112]:
list(range(3))

[0, 1, 2]

In [110]:
# How many data we throw
1 - data_2005.shape[0] / data.shape[0]
# we throw 70% of data

0.6926122050280191

In [16]:
# First filter data from 2005
# 2005-01-01 to 2015-07-01
# For example filter data of Germany vs Spain
df1 = data[(data.home_team == 'Spain') & (data.away_team == 'Germany')]
df2 = data[(data.home_team == 'Germany') & (data.away_team == 'Spain')]
df3 = df1.append(df2).sort_values(by=['date'])
df3.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,id
1741,1935-05-12,Germany,Spain,1,2,Friendly,Cologne,Germany,1741
1822,1936-02-23,Spain,Germany,1,2,Friendly,Barcelona,Spain,1822
2356,1942-04-12,Germany,Spain,1,1,Friendly,Berlin,Germany,2356
3355,1952-12-28,Spain,Germany,2,2,Friendly,Madrid,Spain,3355
4248,1958-03-19,Germany,Spain,2,0,Friendly,Frankfurt am Main,Germany,4248


In [62]:
df3

,date,home_team,away_team,home_score,away_score,tournament,city,country,id
1741,1935-05-12,Germany,Spain,1,2,Friendly,Cologne,Germany,1741
1822,1936-02-23,Spain,Germany,1,2,Friendly,Barcelona,Spain,1822
2356,1942-04-12,Germany,Spain,1,1,Friendly,Berlin,Germany,2356
3355,1952-12-28,Spain,Germany,2,2,Friendly,Madrid,Spain,3355
4248,1958-03-19,Germany,Spain,2,0,Friendly,Frankfurt am Main,Germany,4248
6163,1966-07-20,Germany,Spain,2,1,FIFA World Cup,Birmingham,England,6163
7298,1970-02-11,Spain,Germany,2,0,Friendly,Seville,Spain,7298
8755,1973-11-24,Germany,Spain,2,1,Friendly,Stuttgart,Germany,8755
8820,1974-02-23,Spain,Germany,1,0,Friendly,Barcelona,Spain,8820
9607,1976-04-24,Spain,Germany,1,1,UEFA Euro qualification,Madrid,Spain,9607


In [84]:
# define our own datafram
col_names = ['id','date', 'team_1', 'team_2', 'home_team', 'tournament', 'h_win_diff', 'h_draw', 'f_goalF_1', 
             'f_goalF_2','f_goalA_1','f_goalA_2', 'f_win_1', 'f_win_2', 'f_draw_1', 'f_draw_2','result']
my_df = pd.DataFrame(columns = col_names)
my_df

,id,date,team_1,team_2,home_team,tournament,h_win_diff,h_draw,f_goalF_1,f_goalF_2,f_goalA_1,f_goalA_2,f_win_1,f_win_2,f_draw_1,f_draw_2,result


In [18]:
# Now process in df3 to get desired information
# For example FIFA world cup 2010
df_test = df3.copy()
# Add new column = result to df3
df_test['winner'] = np.where(df_test.home_score > df_test.away_score, df_test.home_team, 
                                 np.where(df_test.home_score < df_test.away_score, df_test.away_team, 'None'))
df_test.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,id,winner
1741,1935-05-12,Germany,Spain,1,2,Friendly,Cologne,Germany,1741,Spain
1822,1936-02-23,Spain,Germany,1,2,Friendly,Barcelona,Spain,1822,Germany
2356,1942-04-12,Germany,Spain,1,1,Friendly,Berlin,Germany,2356,None
3355,1952-12-28,Spain,Germany,2,2,Friendly,Madrid,Spain,3355,None
4248,1958-03-19,Germany,Spain,2,0,Friendly,Frankfurt am Main,Germany,4248,Germany


In [ ]:
# Get ten recent matches - sort and take 10
# df_10 = data[data[home_team] == 'Germany' or ]


date          2008-06-29
home_team        Germany
away_team          Spain
home_score             0
away_score             1
tournament     UEFA Euro
city              Vienna
country          Austria
winner             Spain
Name: 30052, dtype: object

In [19]:
curr_match = df_test.iloc[-4, :]
curr_match

date          2008-06-29
home_team        Germany
away_team          Spain
home_score             0
away_score             1
tournament     UEFA Euro
city              Vienna
country          Austria
id                 30052
winner             Spain
Name: 30052, dtype: object

In [20]:
df_match = df_test[df_test['date'] < curr_match['date'] ]
df_match.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,id,winner
1741,1935-05-12,Germany,Spain,1,2,Friendly,Cologne,Germany,1741,Spain
1822,1936-02-23,Spain,Germany,1,2,Friendly,Barcelona,Spain,1822,Germany
2356,1942-04-12,Germany,Spain,1,1,Friendly,Berlin,Germany,2356,None
3355,1952-12-28,Spain,Germany,2,2,Friendly,Madrid,Spain,3355,None
4248,1958-03-19,Germany,Spain,2,0,Friendly,Frankfurt am Main,Germany,4248,Germany


In [31]:
%config IPCompleter.greedy=True

In [27]:
# Now we do history analysis here
from collections import Counter
count = Counter(df_match['winner']) #Now we get no.win and no.draw
count

Counter({'Germany': 8, 'None': 6, 'Spain': 5})

In [57]:
count['Germany']

8

In [58]:
# Now fill in info in our table
# col_names = ['id','date', 'team_1', 'team_2', 'home_team', 'tournament', 'h_win_diff', 'h_draw', 'f_num_goal_1', 
#              'f_num_goal_2', 'f_win_1', 'f_win_2', 'f_draw_1', 'f_draw_2']
# pre_df = pd.DataFrame(columns = col_names)
# my_df

pre_df = curr_match[['id','date','tournament']].copy()
# 
pre_df['team_1'] =  curr_match['home_team'] if curr_match['home_team'] < curr_match['away_team'] else curr_match['away_team'] 
pre_df['team_2'] =  curr_match['away_team'] if curr_match['home_team'] < curr_match['away_team'] else curr_match['away_team'] 
pre_df['home_team'] = curr_match['home_team']

pre_df['h_win_diff'] = count[pre_df['team_1']] - count[pre_df['team_2']]
pre_df['h_draw'] = count['None']


In [105]:
my_df

,id,date,team_1,team_2,home_team,tournament,h_win_diff,h_draw,f_goalF_1,f_goalF_2,f_goalA_1,f_goalA_2,f_win_1,f_win_2,f_draw_1,f_draw_2,result
30052,30052.0,2008-06-29,Germany,Spain,Germany,UEFA Euro,3.0,6.0,23.0,18.0,12.0,19.0,7.0,7.0,3.0,3.0,lose
31737,31737.0,2010-07-07,Germany,Spain,Germany,FIFA World Cup,2.0,6.0,23.0,18.0,12.0,19.0,7.0,7.0,3.0,3.0,lose
36013,36013.0,2014-11-18,Germany,Spain,Spain,Friendly,1.0,6.0,23.0,18.0,12.0,19.0,7.0,7.0,3.0,3.0,win
38792,38792.0,2018-03-23,Germany,Spain,Germany,Friendly,2.0,6.0,23.0,18.0,12.0,19.0,7.0,7.0,3.0,3.0,draw


In [103]:
curr_match

date          2018-03-23
home_team        Germany
away_team          Spain
home_score             1
away_score             1
tournament      Friendly
city          Düsseldorf
country          Germany
id                 38792
winner              None
Name: 38792, dtype: object

In [97]:
df_test

,date,home_team,away_team,home_score,away_score,tournament,city,country,id,winner
1741,1935-05-12,Germany,Spain,1,2,Friendly,Cologne,Germany,1741,Spain
1822,1936-02-23,Spain,Germany,1,2,Friendly,Barcelona,Spain,1822,Germany
2356,1942-04-12,Germany,Spain,1,1,Friendly,Berlin,Germany,2356,None
3355,1952-12-28,Spain,Germany,2,2,Friendly,Madrid,Spain,3355,None
4248,1958-03-19,Germany,Spain,2,0,Friendly,Frankfurt am Main,Germany,4248,Germany
6163,1966-07-20,Germany,Spain,2,1,FIFA World Cup,Birmingham,England,6163,Germany
7298,1970-02-11,Spain,Germany,2,0,Friendly,Seville,Spain,7298,Spain
8755,1973-11-24,Germany,Spain,2,1,Friendly,Stuttgart,Germany,8755,Germany
8820,1974-02-23,Spain,Germany,1,0,Friendly,Barcelona,Spain,8820,Spain
9607,1976-04-24,Spain,Germany,1,1,UEFA Euro qualification,Madrid,Spain,9607,None


In [104]:
# Assemble
# Before this loop, I need to add winner here
import time

# Drop all data
my_df = my_df[0:0]

# need to get df_test here

start_time = time.time()
# Test history match stat
for i in range(-4,0):
    print(i)
    curr_match = df_test.iloc[i,:]
    df_match = df_test[df_test['date'] < curr_match['date'] ]
    count = Counter(df_match['winner']) #Now we get no.win and no.draw
    
    pre_df = curr_match[['id','date','tournament']].copy()
    
#     	id	date	team_1	team_2	home_team	tournament	h_win_diff	h_draw	f_goalF_1	f_goalF_2	f_goalA_1	f_goalA_2	f_win_1	f_win_2	f_draw_1	f_draw_2	result
    # Get the head-to-head result
    pre_df['team_1'] =  curr_match['home_team'] if curr_match['home_team'] < curr_match['away_team'] else curr_match['away_team'] 
    pre_df['team_2'] =  curr_match['away_team'] if curr_match['home_team'] < curr_match['away_team'] else curr_match['home_team'] 
    pre_df['home_team'] = curr_match['home_team']

    pre_df['h_win_diff'] = count[pre_df['team_1']] - count[pre_df['team_2']]
    pre_df['h_draw'] = count['None']
    
    # Get the recent result of each team
    # Team 1
    team_1 = pre_df['team_1']
    df_10  = data[(data.home_team == team_1) | (data.away_team == team_1)]
    
    length = df_10.shape[0]
    df_10  = df_10.iloc[length-11:length-1,:]
    
    df_10['winner'] = np.where(df_10.home_score > df_10.away_score, df_10.home_team, 
                                     np.where(df_10.home_score < df_10.away_score, df_10.away_team, 'None'))
    
    pre_df['f_goalF_1'] = df_10['home_score'].sum()
    pre_df['f_goalA_1'] = df_10['away_score'].sum()
    
    dict_result_10 = Counter(df_10['winner'])
    pre_df['f_win_1']  = dict_result_10[team_1]
    pre_df['f_draw_1'] = dict_result_10['None']
    
    # Team 2
    team_2 = pre_df['team_2']
    df_10  = data[(data.home_team == team_2) | (data.away_team == team_2)]
    
    length = df_10.shape[0]
    df_10  = df_10.iloc[length-11:length-1,:]
    
    df_10['winner'] = np.where(df_10.home_score > df_10.away_score, df_10.home_team, 
                                     np.where(df_10.home_score < df_10.away_score, df_10.away_team, 'None'))
    
    pre_df['f_goalF_2'] = df_10['home_score'].sum()
    pre_df['f_goalA_2'] = df_10['away_score'].sum()
    
    dict_result_10 = Counter(df_10['winner'])
    pre_df['f_win_2']  = dict_result_10[team_2]
    pre_df['f_draw_2'] = dict_result_10['None']
    
    pre_df['result'] = np.where(curr_match.winner == team_1, 'win', 
                                     np.where(curr_match.winner == team_2, 'lose', 'draw'))
    
    
    my_df = my_df.append(pre_df)
    
print(time.time() - start_time)

-4
-3
-2
-1
2.9774169921875


In [95]:
my_df = my_df[0:0]
my_df

,id,date,team_1,team_2,home_team,tournament,h_win_diff,h_draw,f_goalF_1,f_goalF_2,f_goalA_1,f_goalA_2,f_win_1,f_win_2,f_draw_1,f_draw_2,result


In [89]:
data.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,id
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,0
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,1
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,2
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,3
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,4


In [93]:
#  Ok now move to the next step: retrieve the recent performance 
team_1 = curr_match['home_team']

df_10  = data[(data.home_team == team_1) | (data.away_team == team_1)]
length = df_10.shape[0]
df_10  = df_10.iloc[length-11:length-1,:]
df_10['winner'] = np.where(df_10.home_score > df_10.away_score, df_10.home_team, 
                                 np.where(df_10.home_score < df_10.away_score, df_10.away_team, 'None'))
df_10

# df2 = data[(data.home_team == 'Germany') & (data.away_team == 'Spain')]
# df3 = df1.append(df2).sort_values(by=['date'])
# df_10.iloc[:]

,date,home_team,away_team,home_score,away_score,tournament,city,country,id,winner
38256,2017-06-25,Germany,Cameroon,3,1,Confederations Cup,Sochi,Russia,38256,Germany
38268,2017-06-29,Germany,Mexico,4,1,Confederations Cup,Sochi,Russia,38268,Germany
38279,2017-07-02,Chile,Germany,0,1,Confederations Cup,St Petersburg,Russia,38279,Germany
38378,2017-09-01,Czech Republic,Germany,1,2,FIFA World Cup qualification,Prague,Czech Republic,38378,Germany
38415,2017-09-04,Germany,Norway,6,0,FIFA World Cup qualification,Stuttgart,Germany,38415,Germany
38476,2017-10-05,Northern Ireland,Germany,1,3,FIFA World Cup qualification,Belfast,Northern Ireland,38476,Germany
38528,2017-10-08,Germany,Azerbaijan,5,1,FIFA World Cup qualification,Kaiserslautern,Germany,38528,Germany
38609,2017-11-10,England,Germany,0,0,Friendly,London,England,38609,None
38678,2017-11-14,Germany,France,2,2,Friendly,Cologne,Germany,38678,None
38792,2018-03-23,Germany,Spain,1,1,Friendly,Düsseldorf,Germany,38792,None


In [94]:
df_10['home_score'].sum()
df_10['away_score'].sum()

23

In [96]:
counter = Counter(df_10['winner'])

In [102]:
counter

Counter({'Germany': 7, 'None': 3})

In [101]:
counter['Germany']

7

In [82]:
# df_10.shape
length = df_10.shape[0]

In [92]:
df_10.iloc[length-11:length-1,:]

,date,home_team,away_team,home_score,away_score,tournament,city,country,id,winner
38256,2017-06-25,Germany,Cameroon,3,1,Confederations Cup,Sochi,Russia,38256,Germany
38268,2017-06-29,Germany,Mexico,4,1,Confederations Cup,Sochi,Russia,38268,Germany
38279,2017-07-02,Chile,Germany,0,1,Confederations Cup,St Petersburg,Russia,38279,Germany
38378,2017-09-01,Czech Republic,Germany,1,2,FIFA World Cup qualification,Prague,Czech Republic,38378,Germany
38415,2017-09-04,Germany,Norway,6,0,FIFA World Cup qualification,Stuttgart,Germany,38415,Germany
38476,2017-10-05,Northern Ireland,Germany,1,3,FIFA World Cup qualification,Belfast,Northern Ireland,38476,Germany
38528,2017-10-08,Germany,Azerbaijan,5,1,FIFA World Cup qualification,Kaiserslautern,Germany,38528,Germany
38609,2017-11-10,England,Germany,0,0,Friendly,London,England,38609,None
38678,2017-11-14,Germany,France,2,2,Friendly,Cologne,Germany,38678,None
38792,2018-03-23,Germany,Spain,1,1,Friendly,Düsseldorf,Germany,38792,None


In [76]:
curr_match

date          2018-03-23
home_team        Germany
away_team          Spain
home_score             1
away_score             1
tournament      Friendly
city          Düsseldorf
country          Germany
id                 38792
winner              None
Name: 38792, dtype: object

True